# Data prepping for Age Standarization

***
__Set up__

In [1]:
import pandas as pd

In [2]:
df = pd.read_stata("../data/raw/cps_00009.dta")

print(df.shape)
df.head()

(18363038, 18)


,year,serial,month,hwtfinl,cpsid,asecflag,statefip,county,pernum,wtfinl,cpsidp,age,sex,race,hispan,empstat,labforce,educ
0,2007,1,january,3157.2473,2.006100e+13,NaN,alabama,0,1,3284.6238,2.006100e+13,40,male,black/negro,not hispanic,at work,"yes, in the labor force",grade 10
1,2007,1,january,3157.2473,2.006100e+13,NaN,alabama,0,2,3157.2473,2.006100e+13,39,female,black/negro,not hispanic,at work,"yes, in the labor force",master's degree
2,2007,1,january,3157.2473,2.006100e+13,NaN,alabama,0,3,4597.5894,2.006100e+13,4,female,black/negro,not hispanic,niu,niu,niu or blank
3,2007,1,january,3157.2473,2.006100e+13,NaN,alabama,0,4,2646.0337,2.006100e+13,7,male,black/negro,not hispanic,niu,niu,niu or blank
4,2007,2,january,3778.6089,2.006010e+13,NaN,alabama,0,1,3778.6089,2.006010e+13,55,male,white,not hispanic,at work,"yes, in the labor force","associate's degree, academic program"


In [3]:
# age restriction
df = df[(df['age'] > "15") & (df['age'] < "66")].copy()

In [4]:
condition_advanced = (df['educ'] > "bachelor's degree")
condition_ba = (df['educ'] == "bachelor's degree")
condition_associate = ((df['educ'] == "associate's degree, occupational/vocational program") | (df['educ'] == "associate's degree, academic program"))
condition_somecollege = (df['educ'] == 'some college but no degree')
condition_hs = (df['educ'] == 'high school diploma or equivalent')
condition_lessthanhs = (df['educ'] < 'high school diploma or equivalent')

In [5]:
df.loc[condition_advanced, 'education'] = 'advanced degree'
df.loc[condition_ba, 'education'] = "bachelor's"
df.loc[condition_associate, 'education'] = "associate"
df.loc[condition_somecollege, 'education'] = 'some college'
df.loc[condition_hs, 'education'] = 'high school'
df.loc[condition_lessthanhs, 'education'] = 'less than hs'

# With this we grab our df['educ_attainment'] column and we make it a Categorical series then assign it back to df['educ_attainment']
df['education'] = pd.Categorical(df['education'], categories=['less than hs', 'high school', 'some college', "associate", "bachelor's", 'advanced degree'], ordered = True)

In [6]:
df['empstat'].unique()

[at work, armed forces, nilf, other, has job, not at work last week, nilf, unable to work, nilf, retired, unemployed, experienced worker, unemployed, new worker]
Categories (8, object): [armed forces < at work < has job, not at work last week < unemployed, experienced worker < unemployed, new worker < nilf, unable to work < nilf, other < nilf, retired]

In [7]:
df.loc[((df['empstat'] >= 'at work') & (df['empstat'] <= 'has job, not at work last week')), 'employed'] = 'employed'
df.loc[((df['empstat'] != 'at work') & (df['empstat'] != 'has job, not at work last week')), 'employed'] = 'not employed'

In [8]:
df.head()

,year,serial,month,hwtfinl,cpsid,asecflag,statefip,county,pernum,wtfinl,cpsidp,age,sex,race,hispan,empstat,labforce,educ,education,employed
0,2007,1,january,3157.2473,2.006100e+13,NaN,alabama,0,1,3284.6238,2.006100e+13,40,male,black/negro,not hispanic,at work,"yes, in the labor force",grade 10,less than hs,employed
1,2007,1,january,3157.2473,2.006100e+13,NaN,alabama,0,2,3157.2473,2.006100e+13,39,female,black/negro,not hispanic,at work,"yes, in the labor force",master's degree,advanced degree,employed
4,2007,2,january,3778.6089,2.006010e+13,NaN,alabama,0,1,3778.6089,2.006010e+13,55,male,white,not hispanic,at work,"yes, in the labor force","associate's degree, academic program",associate,employed
5,2007,3,january,1981.4296,2.006121e+13,NaN,alabama,1073,1,1981.4296,2.006121e+13,50,female,black/negro,not hispanic,at work,"yes, in the labor force",some college but no degree,some college,employed
6,2007,3,january,1981.4296,2.006121e+13,NaN,alabama,1073,2,0.0000,2.006121e+13,32,male,black/negro,not hispanic,armed forces,niu,high school diploma or equivalent,high school,not employed


In [9]:
coi = ["year", 'month', 'statefip', 'wtfinl', 'age', 'education', 'employed']

dff = df[coi].copy()

dff.head()

,year,month,statefip,wtfinl,age,education,employed
0,2007,january,alabama,3284.6238,40,less than hs,employed
1,2007,january,alabama,3157.2473,39,advanced degree,employed
4,2007,january,alabama,3778.6089,55,associate,employed
5,2007,january,alabama,1981.4296,50,some college,employed
6,2007,january,alabama,0.0000,32,high school,not employed


In [10]:
dff['age_group'] = pd.cut(df['age'].astype(int), bins = [15,24,34,44,54,65], labels = ['16-24', '25-34', '35-44', '45-54', '55+'])

dff.head()

,year,month,statefip,wtfinl,age,education,employed,age_group
0,2007,january,alabama,3284.6238,40,less than hs,employed,35-44
1,2007,january,alabama,3157.2473,39,advanced degree,employed,35-44
4,2007,january,alabama,3778.6089,55,associate,employed,55+
5,2007,january,alabama,1981.4296,50,some college,employed,45-54
6,2007,january,alabama,0.0000,32,high school,not employed,25-34


In [11]:
total = dff.groupby(['year', 'month', 'education', 'age_group', 'employed'])['wtfinl'].sum().to_frame()
total.rename(columns = {'wtfinl': 'weight'}, inplace = True)

total = total.groupby(['year', 'education', 'age_group', 'employed'])['weight'].mean().to_frame()

In [12]:
total.head()

weight
year education    age_group employed                  
2007 less than hs 16-24     employed      5.096008e+05
                            not employed  1.256748e+06
                  25-34     employed      6.875717e+05
                            not employed  3.500973e+05
                  35-44     employed      7.070350e+05

In [13]:
total_pct = total.groupby(['year', 'education', 'age_group']).apply(lambda x: x/x.sum())
total_pct.rename(columns = {'weight': 'rate'}, inplace = True)
total_pct.head()

rate
year education    age_group employed              
2007 less than hs 16-24     employed      0.288505
                            not employed  0.711495
                  25-34     employed      0.662612
                            not employed  0.337388
                  35-44     employed      0.686302

In [14]:
final = pd.merge(total.reset_index(), total_pct.reset_index()).copy()

final.to_csv("../data/processed/age_adj.csv", index = False,)

final.head()

,year,education,age_group,employed,weight,rate
0,2007,less than hs,16-24,employed,5.096008e+05,0.288505
1,2007,less than hs,16-24,not employed,1.256748e+06,0.711495
2,2007,less than hs,25-34,employed,6.875717e+05,0.662612
3,2007,less than hs,25-34,not employed,3.500973e+05,0.337388
4,2007,less than hs,35-44,employed,7.070350e+05,0.686302


Same process is repeated to create `age_adj_CA.csv` (`dff[dff['statefip'] == 'california']` when creating `total` dataframe)

***
### Reality checks

In [16]:
total.xs(2017, level = 0).sum()

weight    2.091583e+08
dtype: float64

In [17]:
209_158_300

209158300

209,158,300 people in 2017?